In [1]:
from requests import get
from urllib import quote
import pandas as pd
from bs4 import BeautifulSoup
from database_connections import connect_to_postgres
import unidecode
import wikipedia


In [2]:
def create_query_param_string(params):
    param_list = [key+'='+str(value) for key, value in params.items()]
    return '?'+'&'.join(param_list)

def parse_pages_from_json(response_json):
    return response_json['query']['pages']

def parse_headings_from_json(response_json):
    return response_json['mobileview']['sections']

#ADDED
def parse_category_pages_from_json(response_json):
    return response.json()['query']['categorymembers']

def wikipedia_page_format(page):
    return page.lower().capitalize().replace(' ','_')

base_url = "https://en.wikipedia.org/w/api.php"

def wikipedia_get(title, category=False):
    """this function returns the text of a wikipedia page"""
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    
    if category:
        title = "Category:"+title
        
    params['titles'] = wikipedia_page_format(title)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    return response.json()['query']['pages']

def wikipedia_get_category(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    
    params['titles'] = 'Category:'+quote(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    try:
        return parse_pages_from_json(response.json())
    except:
        return response
    
def wikipedia_get_pages_for_category(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'list' : 'categorymembers',
               'cmlimit' : 'max'
             }
    
    params['cmtitle'] = 'Category:'+wikipedia_page_format(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    return response.json()['query']['categorymembers']

# def wikipedia_get_page_headings(title):
#     params = { 'action' : 'mobileview',
#                'format' : 'json',
#                'prop' : 'sections',
#                'sections' : 'all'
#              }
    
#     params['page'] = quote(title)
#     query_param_string = create_query_param_string(params)
#     response = get(base_url+query_param_string)
#     try:
#         return response.json() #parse_headings_from_json(response.json())
#     except:
#         return response

# https://en.wikipedia.org/w
#     /api.php?
#     action=parse&format=json&pageid={}&prop=text%7Csections&contentmodel=wikitext

def wikipedia_get_sections_by_id(pageid):
    """this function returns the text of a wikipedia page"""
    params = { 'action' : 'parse',
               'format' : 'json',
               'prop' : 'sections'
             }
        
    params['pageid'] = pageid
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    return response.json()#['query']['pages']

In [3]:
category = 'Sandwiches'

In [4]:
category_dict = wikipedia_get_category(category)

In [5]:
category_dict

{u'757471': {u'extract': u'',
  u'ns': 14,
  u'pageid': 757471,
  u'title': u'Category:Sandwiches'}}

In [9]:
category_number

757471

In [6]:
category_number = int(unidecode.unidecode(category_dict.keys()[0]))

connection, cursor = connect_to_postgres('remote')
update_sql = """
    INSERT INTO category
    (category_id, category_name) VALUES ({}, '{}');
    """.format(category_number,category)
cursor.execute(update_sql)
connection.commit()
connection.close()

Connected to server joshuacook.me.


In [7]:
pages_list = wikipedia_get_pages_for_category('Sandwiches')
pages_list[:8]

[{u'ns': 0, u'pageid': 82425, u'title': u'Sandwich'},
 {u'ns': 0, u'pageid': 33686134, u'title': u'List of sandwiches'},
 {u'ns': 0, u'pageid': 49033306, u'title': u'Afghani burger'},
 {u'ns': 0, u'pageid': 2546911, u'title': u'Al pastor'},
 {u'ns': 0, u'pageid': 31439892, u'title': u'Bagel toast'},
 {u'ns': 0, u'pageid': 26081358, u'title': u'Bake and Shark'},
 {u'ns': 0, u'pageid': 3334477, u'title': u'Baked bean sandwich'},
 {u'ns': 0, u'pageid': 857888, u'title': u'B\xe1nh m\xec'}]

In [8]:
connection, cursor = connect_to_postgres('remote')

for i in pages_list:
    title = i[u'title']
    if 'Category:' in title:
        continue
    page_id = i[u'pageid']
    title = title.replace("'","")
    html = wikipedia.page(pageid=page_id).content
    html = html.replace("'","")
    print u"Adding {}, id={}".format(title, page_id)
    sections = wikipedia_get_sections_by_id(page_id)
    sections[sections.keys()[0]]['sections']
    section_list = ['']
    for i in sections[sections.keys()[0]]['sections']:
        if i['anchor'] not in ['History','See_also','References','External_links','Gallery']:
            section_list.append(unidecode.unidecode(i['anchor']))
    if len(section_list) > 1:
        section_list.pop(0)
    insert_page = u"""
        INSERT INTO page (page_id, title, page, section_head)
        VALUES ({}, '{}', '{}', ARRAY{});
        """.format(page_id, title, html, section_list)
    insert_page_cate = u"""
        INSERT INTO page_cate (page_id, category_id)
        VALUES ({}, {});
        """.format(page_id, category_number)
    cursor.execute(insert_page)
    cursor.execute(insert_page_cate)
    connection.commit()

cursor.close()
connection.close()


Connected to server joshuacook.me.
Adding Sandwich, id=82425


IntegrityError: null value in column "category_id" violates not-null constraint
DETAIL:  Failing row contains (82425, null, Sandwich, 2017-02-03 00:43:36.914328, A sandwich is a food item consisting of one or more types of foo..., {Etymology,Usage,Varieties}).


# NOW UPLOAD TO DATABASE!!!!

### then abstract to a module.


In [ ]:
page_dict = wikipedia_get('Afghani burger')

In [ ]:
pageid = int(unidecode.unidecode(page_dict.keys()[0]))
page = BeautifulSoup(page_dict[str(pageid)][u'extract'], "html.parser").get_text()


In [ ]:
def wikipedia_get(title, category=False):
    """this function returns the text of a wikipedia page"""
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    
    if category:
        title = "Category:"+title
        
    params['titles'] = wikipedia_page_format(title)
    
    query_param_string = create_query_param_string(params)
    
    response = get(base_url+query_param_string)
    
    return response.json()['query']['pages']

In [ ]:
a = wikipedia_get('Lunch')
b = a['18589377']['extract']

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(b, "html.parser")
soup.get_text()

In [ ]:
wikipedia_get_category('Automotive technologies')

In [ ]:
wiki_pages_automotive_techs = wikipedia_get_pages_for_category('Automotive technologies')
wiki_pages_automotive_techs = wiki_pages_automotive_techs[1:]
wiki_pages_automotive_techs

In [ ]:
ID = [wiki_pages_automotive_techs[i]['pageid'] for i in range(len(wiki_pages_automotive_techs))]
title = [wiki_pages_automotive_techs[i]['title'] for i in range(len(wiki_pages_automotive_techs))]
ID_title = zip(ID, title)

In [ ]:
ID_title

In [ ]:
wiki_pages_automotive_techs_df = pd.DataFrame(wiki_pages_automotive_techs)
wiki_pages_automotive_techs_df

In [ ]:
len(wiki_pages_automotive_techs)

In [ ]:
type(wiki_pages_automotive_techs)

### Sections/Headings

In [ ]:
def wikipedia_get_page_headings(title):
    params = { 'action' : 'mobileview',
               'format' : 'json',
               'prop' : 'sections',
               'sections' : 'all'
             }
    
    params['page'] = quote(title)
    
    query_param_string = create_query_param_string(params)
    
    response = get(base_url+query_param_string)
    
    try:
        return response.json() #parse_headings_from_json(response.json())
    except:
        return response

In [ ]:
wiki_pages_headings_automotive_techs = {}
for i in wiki_pages_automotive_techs_df['title']:
    b = wikipedia_get_page_headings(i)
    wiki_pages_headings_automotive_techs[i] = b
# [wikipedia_get_page_headings(title) for title in wiki_pages_automotive_techs_df['title']]

In [ ]:
wiki_pages_headings_automotive_techs

In [ ]:
def wikipedia_get_page_headings2(title):
    params = { 'action' : 'mobileview',
               'format' : 'json',
               'prop' : 'sections',
               'sections' : 'all'
             }
    
    params['page'] = quote(title)
    
    query_param_string = create_query_param_string(params)
    
    response = get(base_url+query_param_string)
    
    try:
        return parse_headings_from_json(response.json())
    except:
        return response

In [ ]:
wiki_pages_headings_automotive_techs2 = {}
for i in wiki_pages_automotive_techs_df['title']:
    b = wikipedia_get_page_headings2(i)
    wiki_pages_headings_automotive_techs2[i] = b
wiki_pages_headings_automotive_techs2

In [ ]:
wiki_categid_section = {}
for l in range(len(ID_title)):
    wiki_categid_section[ID_title[l][0]] = wiki_pages_headings_automotive_techs[ID_title[l][1]]
wiki_categid_section

In [ ]:
wiki_categid_section_S = []
row = {}
for categid, categ in ID_title:
    categdd = wiki_categid_section[categid]['mobileview']['sections'][1:]
    for x in range(len(categdd)):
        row = {'categ_id':categid, 'section_id':categdd[x]['id'], 'section_name':categdd[x]['line']}
        wiki_categid_section_S.append(row)
wiki_categid_section_S

In [ ]:
wiki_categid_section_df = pd.DataFrame(wiki_categid_section_S)
wiki_categid_section_df

In [ ]:
import requests
response = get("https://en.wikipedia.org/?curid=7446779")
test_text = response.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(test_text)

In [ ]:
soup

In [ ]:
# >>> r = requests.get('https://api.github.com')
# >>> r.status_code
# 200
# >>> r.headers['content-type']
# 'application/json; charset=utf8'
# >>> r.encoding
# 'utf-8'
# >>> r.text
# u'{"type":"User"...'
# >>> r.json()
# {u'private_gists': 419, u'total_private_repos': 77, ...}

In [ ]:
import requests
response = get("https://en.wikipedia.org/wiki/AC_Propulsion_eBox")
response.text

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia
a = wikipedia.page("AC Propulsion eBox")
a.content

In [ ]:
wiki_pages_automotive_techs_df[\
                               wiki_pages_automotive_techs_df['title']=="AC Propulsion eBox"]["pageid"]

In [ ]:
import numpy
import psycopg2


In [ ]:
con, cur = connect_to_postgres()
insert_page_vector_sql = """
    INSERT INTO page_vector (page_id, encoded_vector)
    VALUES ({},{})""".format(page_id, psycopg2.Binary(---VECTOR----))

cursor.execute(insert_page_vector_sql)
connection.commit()

cursor.close()
connection.close()

In [ ]:
html = wikipedia.page('Category:Sandwiches')

In [ ]:
html.pageid
